<a href="https://colab.research.google.com/github/nxlr/IDLResearchTask/blob/main/Python_Sachin_Sharma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of required packages
required_packages = [
    "numpy", "pandas", "matplotlib", "fuzzywuzzy", "python-Levenshtein", "tqdm" # Add your required packages here
]

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        install(package)


# Import modules after they have been checked for installation
import os
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm

python-Levenshtein not found. Installing...


In [13]:
# Load the datasets
foreign_names = pd.read_csv('/content/ForeignNames_2019_2020.csv')
country_iso = pd.read_csv('/content/Country_Name_ISO3.csv')

# Merge the datasets on the country name columns
merged_data = pd.merge(foreign_names, country_iso, left_on='foreigncountry_cleaned', right_on='country_name', how='left')

# Manually assign the ISO3 code and country name for "South Korea"
merged_data.loc[merged_data['foreigncountry_cleaned'] == 'South Korea', 'country_iso3'] = 'KOR'
merged_data.loc[merged_data['foreigncountry_cleaned'] == 'South Korea', 'country_name'] = 'South Korea'

# Save the merged dataset for future use
merged_data.to_csv('/content/merged_ForeignNames_2019_2020.csv', index=False)

# print(merged_data.head())

In [14]:
# Load the merged dataset
merged_data = pd.read_csv('/content/merged_ForeignNames_2019_2020.csv')

# Clean the firm names by lowercasing and stripping whitespace
merged_data['cleaned_firm_name'] = merged_data['foreign'].str.lower().str.strip()

# Convert all entries to strings and replace NaN values with an empty string
merged_data['cleaned_firm_name'] = merged_data['cleaned_firm_name'].fillna('').astype(str)

In [15]:
# Group the data by country to perform matching within each country
def standardize_names(group):
    # Get unique firm names and filter out any empty strings
    unique_names = [name for name in group['cleaned_firm_name'].unique() if name]

    # Create a dictionary to store the best matches
    best_match_dict = {}

    # Iterate through each unique name
    for name in unique_names:
        # If name is already matched, skip it
        if name in best_match_dict:
            continue

        # Find top 3 best matches for the name within the same group
        matches = process.extract(name, unique_names, scorer=fuzz.token_sort_ratio, limit=3)

        # Assign the best match name to all similar names
        for match_name, score in matches:
            if score > 85:  # Set a threshold for similarity
                best_match_dict[match_name] = name

    # Apply the best matches to the group
    group['standardized_name'] = group['cleaned_firm_name'].map(best_match_dict)

    return group

# Process data in smaller chunks
chunk_size = 5000  # Define a chunk size
chunks = [merged_data[i:i + chunk_size] for i in range(0, merged_data.shape[0], chunk_size)]

# Initialize an empty list to collect results
standardized_chunks = []

# Process each chunk individually with a progress bar
for chunk in tqdm(chunks, desc="Processing Chunks"):
    standardized_chunk = chunk.groupby('foreigncountry_cleaned').apply(standardize_names)
    standardized_chunks.append(standardized_chunk)

# Combine all chunks into a single DataFrame
standardized_data = pd.concat(standardized_chunks)

Processing Chunks: 100%|██████████| 125/125 [1:00:17<00:00, 28.94s/it]


In [29]:

# Reset the index to remove ambiguity
standardized_data = standardized_data.reset_index(drop=True)

# Assign unique IDs based on the standardized names within each country
standardized_data['cleaned_ID'] = standardized_data.groupby(['foreigncountry_cleaned', 'standardized_name']).ngroup().apply(lambda x: f"{x+1:06}")
standardized_data['cleaned_ID'] = standardized_data['country_iso3'] + standardized_data['cleaned_ID']

# Remove rows with NaN values in important columns
standardized_data = standardized_data.dropna(subset=['foreign', 'standardized_name', 'cleaned_ID'])

# Display the updated dataset
print(standardized_data[['foreign', 'standardized_name', 'cleaned_ID']].head())

# Save the standardized dataset to a CSV file
standardized_data.to_csv('/content/outputfile_sachin_1.csv', index=False)

# Create a file with only the firms whose names have changed
changed_names = standardized_data[standardized_data['foreign'] != standardized_data['standardized_name']]

# Remove rows with NaN values in the changed data
changed_names = changed_names.dropna(subset=['foreign', 'standardized_name'])

changed_names[['foreign', 'standardized_name', 'cleaned_ID']].to_csv('/content/outputfile_sachin_1_changed.csv', index=False)


                       foreign            standardized_name cleaned_ID
0                Inayat Sheraz                inayat sheraz  AFG000064
1                 Ijaz Shakeel                 ijaz shakeel  AFG000059
2                 Shahed Hakim                 shahed hakim  AFG000120
3  Lg Electronics Algerie Sarl  lg electronics algerie sarl  DZA000341
4         Toyota Argentina S A         toyota argentina s a  ARG001676


In [27]:
print(standardized_data.head())
len(standardized_data)

                       foreign foreigncountry_cleaned  shpmtyear country_name  \
0                Inayat Sheraz            Afghanistan       2020  Afghanistan   
1                 Ijaz Shakeel            Afghanistan       2020  Afghanistan   
2                 Shahed Hakim            Afghanistan       2020  Afghanistan   
4  Lg Electronics Algerie Sarl                Algeria       2020      Algeria   
5         Toyota Argentina S A              Argentina       2020    Argentina   

  country_iso3            cleaned_firm_name            standardized_name  \
0          AFG                inayat sheraz                inayat sheraz   
1          AFG                 ijaz shakeel                 ijaz shakeel   
2          AFG                 shahed hakim                 shahed hakim   
4          DZA  lg electronics algerie sarl  lg electronics algerie sarl   
5          ARG         toyota argentina s a         toyota argentina s a   

  cleaned_ID  
0  AFG0064.0  
1  AFG0059.0  
2  AFG0120.

621437

In [26]:
print(changed_names[['foreign', 'standardized_name', 'cleaned_ID']].head())
len(changed_names)

                       foreign            standardized_name cleaned_ID
0                Inayat Sheraz                inayat sheraz  AFG0064.0
1                 Ijaz Shakeel                 ijaz shakeel  AFG0059.0
2                 Shahed Hakim                 shahed hakim  AFG0120.0
4  Lg Electronics Algerie Sarl  lg electronics algerie sarl  DZA0341.0
5         Toyota Argentina S A         toyota argentina s a  ARG1693.0


621406

In [30]:
from google.colab import files
files.download('/content/outputfile_sachin_1.csv')
files.download('/content/outputfile_sachin_1_changed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>